<a href="https://colab.research.google.com/github/Marieta3/sentiment-analysis-russian/blob/master/sentiment_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analiza sentimenta tvitova na ruskom jeziku


##Importovanje potrebnih biblioteka

In [0]:
import tensorflow as tf
import tflearn
import pandas as pd
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer
import re
from collections import Counter
from datetime import datetime
import numpy as np
from tflearn.data_utils import to_categorical

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

##Import za custom nn

In [0]:
from __future__ import print_function

from abc import abstractmethod
import math
import random
import copy

##Konstante

In [0]:
VOCAB_SIZE = 5000
MANJI_DEO = True
MANJI_RANDOM = False
TEST_SIZE = 0.3
KOLICINA = 50000

##Učitavanje podataka

In [0]:
col = 3 #kolona koja sadrzi tvit, ostale kolone nisu potrebne

if(MANJI_DEO):
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]].head(KOLICINA);
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]].head(KOLICINA);
elif (MANJI_RANDOM):
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]].sample(KOLICINA);
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]].sample(KOLICINA);
else:
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]];
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]];  

##Testiranje učitanih podataka


In [120]:
print('Pozitivnih tvitova: ' +str(len(positive)))
print('Negativnih tvitova: ' +str(len(negative)))


Pozitivnih tvitova: 50000
Negativnih tvitova: 50000


##Izvlačenje osnovnog oblika iz tokena

In [121]:
stemmer = RussianStemmer()
stem_cache={}  #smanjuje vreme izvršavanja, ako se reč već pojavljivala, samo je uzmemo iz liste
def get_stem(token):
  stem =stem_cache.get(token, None)
  if stem:
    return stem
  
  token=re.sub('[^а-яА-Я ]', '', token)  #samo slova
  stem=stemmer.stem(token)
  stem_cache[token]=stem
  return stem
get_stem('!')

''

##Tokenizacija

In [0]:
recnik = Counter()
tokenizer = TweetTokenizer()
stop_words = stopwords.words('russian')

def popuni_recnik(tweets):
  for t in tweets.values.tolist():
    #print(t[0])
    tweet=t[0]
    
    tokens=tokenizer.tokenize(tweet)
    for token in tokens:
      stem=get_stem(token)
      if stem!='' and stem not in stop_words and token not in stop_words:
        recnik[stem]+=1
    
start_time=datetime.now()  
popuni_recnik(positive)
popuni_recnik(negative)
end_time=datetime.now()

In [123]:
print('Потребно време за попуњавање речника: ', end_time-start_time)
print("Укупан број различитих речи: ", len(recnik))


Потребно време за попуњавање речника:  0:00:50.099191
Укупан број различитих речи:  56251


In [0]:
vocab = sorted(recnik, key=recnik.get, reverse=True)[:VOCAB_SIZE]  #сортирано по броју понављања опадајуће, првих 5000

In [179]:
print(vocab[:10])
print(vocab[1000:1020])
print(vocab[-10:])
print("Дужина скраћеног речника: ",len(vocab))

['эт', 'сегодн', 'прост', 'мо', 'хоч', 'очен', 'ден', 'теб', 'буд', 'дом']
['выигра', 'матч', 'непонятн', 'детск', 'взросл', 'ждем', 'помо', 'всегд', 'пьян', 'губ', 'какиет', 'совершен', 'познаком', 'чуд', 'мнен', 'лев', 'парол', 'созда', 'осен', 'наоборот']
['нагрузк', 'спасл', 'гриш', 'потянул', 'креативн', 'кран', 'уголовн', 'доброт', 'футбольн', 'отреза']
Дужина скраћеног речника:  5000


In [180]:
idx=5000-1
print("Reč: ", vocab[idx], " se pojavila ", recnik.get(vocab[idx]), " puta")




Reč:  отреза  se pojavila  15  puta


In [0]:
token_to_idx = {vocab[i] : i for i in range(VOCAB_SIZE)}
print(token_to_idx.get('', None))
print(token_to_idx)

##Pretvaranje tvita u vektor

*   vektor je duzine broja reci u recniku (5000)
*   ukoliko tvit sadrzi rec koja se nalazi u recniku, na odgovarajucem mestu u vektoru se nalazi jedinica; u suprotnom nula


In [0]:
def tweet_to_vector(tweet, print_unknown=False):
  vector=np.zeros(VOCAB_SIZE, dtype=int)
  for token in tokenizer.tokenize(tweet):
    stem=get_stem(token)
    idx=token_to_idx.get(stem, None)
    if idx is not None:
      vector[idx]=1
    else:
      if(print_unknown):
        print('Nepoznata rec: ', token)
  return vector
    

In [0]:
def tweet_to_vector1(tweet, print_unknown=False):
  tokenized=tokenizer.tokenize(tweet)
  
  #vector=np.zeros(len(tokenized), dtype=int)
  vector=[0]*len(tokenized)
  cnt=0
  for token in tokenized:
    stem=get_stem(token)
    idx=token_to_idx.get(stem, None)
    if idx is not None:
      vector[cnt]=idx
      cnt+=1
    else:
      cnt+=1
      if(print_unknown):
        print('Nepoznata rec: ', token)
  return vector



In [183]:
tw_idx=1
pos_tws=positive.values.tolist()
neg_tws=negative.values.tolist()
print("tweet: {}".format(pos_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(pos_tws[tw_idx][0])[:10]))
print("tweet: {}".format(neg_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(neg_tws[tw_idx][0])[:10]))

tweet: Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
tweet: [0 0 0 1 0 0 0 0 0 0]
tweet: Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(
tweet: [0 0 0 0 0 0 0 0 0 0]


In [0]:
try:
  del train_vectors
  del test_vectors
except:
  print('Nije definisano')
  
train_size = round((len(negative) + len(positive))*(1-TEST_SIZE))
test_size = len(negative) + len(positive) - train_size


In [0]:
train_vectors = np.zeros((train_size, VOCAB_SIZE), dtype=np.int_)
test_vectors = np.zeros((test_size, VOCAB_SIZE), dtype=np.int_)

In [0]:
tweets=[]
cnt=0
cnt2=0

train_pos_size = round(len(positive)*(1-TEST_SIZE))
train_neg_size = round(len(negative)*(1-TEST_SIZE))

for tweet in negative.values.tolist():
  #print(tweet[0])
  tweets.append(tweet[0])
  if cnt < train_neg_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1
for tweet in positive.values.tolist():
  tweets.append(tweet[0])
  if cnt < train_neg_size + train_pos_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1


In [187]:
print(len(train_vectors))
print(len(train_vectors[0]))
print(test_vectors[:2])

70000
5000
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


##Brisanje vektora tvitova

In [0]:
del train_vectors
del test_vectors

##Labele

In [0]:
#labels = np.append(np.zeros(len(negative), dtype=np.int_), np.ones(len(positive), dtype=np.int_))

train_labels = np.append(
    np.zeros(train_neg_size, dtype=np.int_), 
    np.ones(train_pos_size, dtype=np.int_))

test_labels = np.append(
    np.zeros(len(negative) - train_neg_size, dtype=np.int_), 
    np.ones(len(positive) - train_pos_size, dtype=np.int_))

In [0]:
y_train = to_categorical(train_labels, 2)
y_test = to_categorical(test_labels, 2)


In [190]:
train_labels[50000]

1

##Kreiranje NN modela

In [0]:
def build_model(learning_rate=0.1):
    tf.reset_default_graph()
    
    net = tflearn.input_data([None, VOCAB_SIZE])
    net = tflearn.fully_connected(net, 125, activation='ReLU')
    net = tflearn.fully_connected(net, 25, activation='ReLU')
    net = tflearn.fully_connected(net, 2, activation='softmax')
    regression = tflearn.regression(
        net, 
        optimizer='sgd', 
        learning_rate=learning_rate, 
        loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [0]:

model = build_model(learning_rate=0.75)

In [193]:
model.fit(
    train_vectors, 
    y_train, 
    validation_set=0.1, 
    show_metric=True, 
    batch_size=128, 
    n_epoch=3)

Training Step: 1478  | total loss: 0.50317 | time: 5.174s
| SGD | epoch: 003 | loss: 0.50317 - acc: 0.7473 -- iter: 62976/63000
Training Step: 1479  | total loss: 0.49778 | time: 6.243s
| SGD | epoch: 003 | loss: 0.49778 - acc: 0.7499 | val_loss: 0.58318 - val_acc: 0.6933 -- iter: 63000/63000
--


##Kreiranje RNN modela

In [0]:
train_vectors = [0]*train_size
test_vectors = [0]*test_size


In [0]:
tweets=[]
cnt=0
cnt2=0

train_pos_size = round(len(positive)*(1-TEST_SIZE))
train_neg_size = round(len(negative)*(1-TEST_SIZE))

for tweet in negative.values.tolist():
  #print(tweet[0])
  tweets.append(tweet[0])
  if cnt < train_neg_size:
    train_vectors[cnt]=tweet_to_vector1(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector1(tweet[0])
    cnt2+=1
for tweet in positive.values.tolist():
  tweets.append(tweet[0])
  if cnt < train_neg_size + train_pos_size:
    train_vectors[cnt]=tweet_to_vector1(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector1(tweet[0])
    cnt2+=1


In [0]:
num_tokens = [len(tokens) for tokens in train_vectors + test_vectors]
num_tokens = np.array(num_tokens)

In [144]:
np.mean(num_tokens)

16.14727

In [145]:
np.max(num_tokens)

63

In [146]:

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

28

In [147]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94413

In [0]:
pad = 'pre'
x_train_pad = pad_sequences(train_vectors, maxlen=max_tokens,
                            padding=pad, truncating=pad)

x_test_pad = pad_sequences(test_vectors, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [0]:
def build_RNN():
  model = Sequential()
  model.add(Embedding(input_dim=VOCAB_SIZE,
                    output_dim=8,
                    input_length=max_tokens,
                    name='layer_embedding'))
  model.add(GRU(units=16, return_sequences=True))
  model.add(GRU(units=8, return_sequences=True))
  model.add(GRU(units=4))
  model.add(Dense(1, activation='sigmoid'))
  
  return model

In [0]:
model_rnn=build_RNN()
optimizer = Adam(lr=1e-3)
model_rnn.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [174]:
model_rnn.fit(x_train_pad, train_labels,
          validation_split=0.05, epochs=3, batch_size=128)

Train on 66500 samples, validate on 3500 samples
Epoch 1/3
66500/66500 [==============================] - 99s 1ms/sample - loss: 0.6063 - acc: 0.6525 - val_loss: 0.6382 - val_acc: 0.6446
Epoch 2/3
66500/66500 [==============================] - 91s 1ms/sample - loss: 0.5308 - acc: 0.7342 - val_loss: 0.6493 - val_acc: 0.6537
Epoch 3/3
66500/66500 [==============================] - 96s 1ms/sample - loss: 0.5166 - acc: 0.7447 - val_loss: 0.7023 - val_acc: 0.6026


In [0]:
result = model_rnn.evaluate(x_test_pad, test_labels)

In [176]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 68.25%


##Testiranje

In [194]:
predictions = (np.array(model.predict(test_vectors))[:,0] >= 0.5).astype(np.int_)
accuracy = np.mean(predictions == y_test[:,0], axis=0)
print("Accuracy: ", accuracy)

Accuracy:  0.6844


In [0]:
def test_tweet(tweet):
  
    tweet_vector = tweet_to_vector(tweet, False)
    
    positive_prob = model.predict([tweet_vector])[0][1]
    
    
    print('Original tweet: {}'.format(tweet))
    print('P(positive) = {:.5f}. Result: '.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')

In [0]:
def test_tweet_number(idx):
    test_tweet(tweets[idx])

In [203]:
test_tweet_number(-1)

Original tweet: так вот я сижу с идиотской улыбкой на лице...и смотрю 3 сезон)))
P(positive) = 0.89691. Result:  Positive


##Proba

In [0]:
tweets_for_testing = [
    "меня оштрафовали по дороге домой!",
    "Я люблю мороженое!",
    "Я не люблю мороженое",
    "меня обманули",
    "меня обманули позитив",
    "меня он очень бесить",
    "меня он не бесить",
    "Почему по ночам мне не спится? У машины скрипит колесо. Отразилась луна в ягодице. 10 лет тебе,синяя птица! Жаль,что букв всего лишь сто со",
    "Смотрите сегодня, в 23:35, сразу после финала #голос6 @voice1tv, на @1tv, большой новогодний выпуск @vecherniy_urgant #вечернийургант! Я, @urgantcom и @lazarevsergey приготовили для вас праздничный новогодний номер!))) #иванургант #сергейлазарев #билан #… http://ift.tt/2zLgzWk ",
    "ШОК! Данилу Козловского задержали в центре Москвы за чтение Шекспира!",
    "Через час в Петербурге,на улице Рубинштейна,дом 23 откроют памятник Сергею Довлатову.Надеваем халаты,тапочки,берем собак и цветы!",
    "Спасибо академикам,коллегам и номинантам!Спасибо всей команде ВУ!Спасибо моей семье!И конечно спасибо Вам,дорогие телезрители!#тэфи2016",
    "А вот и ласточка-новостнуха!24 апреля в Крокусе я поделюсь с Вами инсайтом на Synergy Insight Forum. Приходите что ли! #sif2017"
]


##NN testiranje

In [205]:
for tweet in tweets_for_testing:
    test_tweet(tweet) 
    print("---------")

Original tweet: меня оштрафовали по дороге домой!
P(positive) = 0.25807. Result:  Negative
---------
Original tweet: Я люблю мороженое!
P(positive) = 0.82409. Result:  Positive
---------
Original tweet: Я не люблю мороженое
P(positive) = 0.82409. Result:  Positive
---------
Original tweet: меня обманули
P(positive) = 0.30835. Result:  Negative
---------
Original tweet: меня обманули позитив
P(positive) = 0.40771. Result:  Negative
---------
Original tweet: меня он очень бесить
P(positive) = 0.09433. Result:  Negative
---------
Original tweet: меня он не бесить
P(positive) = 0.11769. Result:  Negative
---------
Original tweet: Почему по ночам мне не спится? У машины скрипит колесо. Отразилась луна в ягодице. 10 лет тебе,синяя птица! Жаль,что букв всего лишь сто со
P(positive) = 0.20312. Result:  Negative
---------
Original tweet: Смотрите сегодня, в 23:35, сразу после финала #голос6 @voice1tv, на @1tv, большой новогодний выпуск @vecherniy_urgant #вечернийургант! Я, @urgantcom и @lazarev

##RNN testiranje

In [177]:
tweet_vectors = [0]*len(tweets_for_testing)# = tweet_to_vector1(tweet, False)
for i in range(len(tweets_for_testing)):
  tweet_vectors[i]=tweet_to_vector1(tweets_for_testing[i])

tokens_pad = pad_sequences(tweet_vectors, maxlen=max_tokens,
                       padding=pad, truncating=pad)

model_rnn.predict(tokens_pad)

array([[0.2563761 ],
       [0.71415895],
       [0.6430245 ],
       [0.25210702],
       [0.40163317],
       [0.14506781],
       [0.16822496],
       [0.1695342 ],
       [0.87611693],
       [0.29174778],
       [0.90626216],
       [0.92793053],
       [0.32408455]], dtype=float32)

##Custom neuronska mreza

In [0]:

random.seed(1337)

class ComputationalNode(object):

    @abstractmethod
    def forward(self, x):
        pass

    @abstractmethod
    def backward(self, dz):
        pass
      
class MultiplyNode(ComputationalNode):

    def __init__(self):
        self.x = [0., 0.]  # x[0] je ulaz, x[1] je tezina

    def forward(self, x):
        self.x = x
        # TODO 1: implementirati forward-pass za mnozac
        #raise NotImplementedError('TODO 1')
        return self.x[0]*self.x[1]

    def backward(self, dz):
        # TODO 1: implementirati backward-pass za mnozac
        #raise NotImplementedError('TODO 1')
        return [dz*self.x[1], dz*self.x[0]]
      
class SumNode(ComputationalNode):

    def __init__(self):
        self.x = []  # x je vektor, odnosno niz skalara

    def forward(self, x):
        self.x = x
        # TODO 2: implementirati forward-pass za sabirac
        #raise NotImplementedError('TODO 2')
        return sum(self.x)

    def backward(self, dz):
        # TODO 2: implementirati backward-pass za sabirac
        #raise NotImplementedError('TODO 2')
        return [dz]*len(self.x)
      


In [0]:
class SigmoidNode(ComputationalNode):

    def __init__(self):
        self.x = 0.  # x je skalar

    def forward(self, x):
        self.x = x
        # TODO 3: implementirati forward-pass za sigmoidalni cvor
        #raise NotImplementedError('TODO 3')
        return self._sigmoid(self.x)

    def backward(self, dz):
        # TODO 3: implementirati backward-pass za sigmoidalni cvor
        #raise NotImplementedError('TODO 3')
        return dz*self._sigmoid(self.x)*(1-self._sigmoid(self.x))

    def _sigmoid(self, x):
        # TODO 3: implementirati sigmoidalnu funkciju
        #raise NotImplementedError('TODO 3')
        return 1/(1+math.exp(-x))

In [0]:
class ReLuNode(ComputationalNode):

    def __init__(self):
        self.x = 0.  # x je skalar

    def forward(self, x):
        self.x = x
        return self._relu(self.x)

    def backward(self, dz):
        if self.x > 0.:
            return dz*1.
        else:
            return dz*0.

    def _relu(self, x):
        return max(0., x)

In [0]:
class NeuronNode(ComputationalNode):

    def __init__(self, n_inputs, activation):
        self.n_inputs = n_inputs  # moramo da znamo kolika ima ulaza da bismo znali koliko nam treba mnozaca
        self.multiply_nodes = []  # lista mnozaca
        self.sum_node = SumNode()  # sabirac

        # TODO 4: napraviti n_inputs mnozaca u listi mnozaca, odnosno mnozac za svaki ulaz i njemu odgovarajucu tezinu
        # za svaki mnozac inicijalizovati tezinu na broj iz normalne (gauss) raspodele sa st. devijacijom 0.1
        for n in range(n_inputs):
            mn=MultiplyNode()
            mn.x=[1., random.gauss(0., 0.1)]
            self.multiply_nodes.append(mn)

        # TODO 5: dodati jos jedan mnozac u listi mnozaca, za bias
        # bias ulaz je uvek fiksiran na 1.
        # bias tezinu inicijalizovati na broj iz normalne (gauss) raspodele sa st. devijacijom 0.01
        bias=MultiplyNode()
        bias.x=[1., random.gauss(0., 0.01)]
        self.multiply_nodes.append(bias)

        # TODO 6: ako ulazni parametar funckije 'activation' ima vrednosti 'sigmoid',
        # inicijalizovati da aktivaciona funckija bude sigmoidalni cvor
        self.activation_node = None
        if activation=='sigmoid':
            self.activation_node=SigmoidNode()
        elif activation=='relu':
            self.activation_node=ReLuNode()
        self.previous_deltas=[0.]*(self.n_inputs+1)

        self.gradients = []

    def forward(self, x):  # x je vektor ulaza u neuron, odnosno lista skalara
        #x = copy.copy(x)
        x=np.append(x, 1.)  # uvek implicitino dodajemo bias=1. kao ulaz

        # TODO 7: implementirati forward-pass za vestacki neuron
        # u x se nalaze ulazi i bias neurona
        # iskoristi forward-pass za mnozace, sabirac i aktivacionu funkciju da bi se dobio konacni izlaz iz neurona
        for_sum=[]
        for i, xx in enumerate(x):
            inp=[x[i], self.multiply_nodes[i].x[1]]
            for_sum.append(self.multiply_nodes[i].forward(inp))
        summed=self.sum_node.forward(for_sum)
        summed_act=self.activation_node.forward(summed)
        return summed_act

    def backward(self, dz):
        dw = []
        d = dz[0] if type(dz[0]) == float else sum(dz)  # u d se nalazi spoljasnji gradijent izlaza neurona

        # TODO 8: implementirati backward-pass za vestacki neuron
        # iskoristiti backward-pass za aktivacionu funkciju, sabirac i mnozace da bi se dobili gradijenti tezina neurona
        # izracunate gradijente tezina ubaciti u listu dw
        d=self.activation_node.backward(d)
        d=self.sum_node.backward(d)
        for i, dd in enumerate(d):
            dw.append(self.multiply_nodes[i].backward(dd)[1])
        self.gradients.append(dw)
        return dw

    def update_weights(self, learning_rate, momentum):
        # azuriranje tezina vestackog neurona
        # learning_rate je korak gradijenta

        # TODO 11: azurirati tezine neurona (odnosno azurirati drugi parametar svih mnozaca u neuronu)
        # gradijenti tezina se nalaze u list self.gradients
        #raise NotImplementedError('TODO 11')
        for i, multiply_node in enumerate(self.multiply_nodes):
            mean_gradient=sum([grad[i] for grad in self.gradients])/len(self.gradients)
            delta=learning_rate*mean_gradient+momentum*self.previous_deltas[i]
            self.previous_deltas[i]=delta
            self.multiply_nodes[i].x[1]-=delta
        self.gradients = []  # ciscenje liste gradijenata (da sve bude cisto za sledecu iteraciju)

In [0]:
class NeuralLayer(ComputationalNode):

    def __init__(self, n_inputs, n_neurons, activation):
        self.n_inputs = n_inputs  # broj ulaza u ovaj sloj neurona
        self.n_neurons = n_neurons  # broj neurona u sloju (toliko ce biti i izlaza iz ovog sloja)
        self.activation = activation  # aktivaciona funkcija neurona u ovom sloju

        self.neurons = []
        # konstruisanje sloja nuerona
        for _ in range(n_neurons):
            neuron = NeuronNode(n_inputs, activation)
            self.neurons.append(neuron)

    def forward(self, x):  # x je vektor, odnosno lista "n_inputs" elemenata
        layer_output = []
        # forward-pass za sloj neurona je zapravo forward-pass za svaki neuron u sloju nad zadatim ulazom x
        for neuron in self.neurons:
            neuron_output = neuron.forward(x)
            layer_output.append(neuron_output)

        return layer_output

    def backward(self, dz):  # dz je vektor, odnosno lista "n_neurons" elemenata
        dd = []
        # backward-pass za sloj neurona je zapravo backward-pass za svaki neuron u sloju nad
        # zadatim spoljasnjim gradijentima dz
        print(dz[0])
        print(len(dz))
        print(enumerate(self.neurons))
        for i, neuron in enumerate(self.neurons):
            neuron_dz = [d[i] for d in dz]
            neuron_dz = neuron.backward(neuron_dz)
            dd.append(neuron_dz[:-1])  # izuzimamo gradijent za bias jer se on ne propagira unazad
        return dd

    def update_weights(self, learning_rate, momentum):
        # azuriranje tezina slojeva neurona je azuriranje tezina svakog neurona u tom sloju
        for neuron in self.neurons:
            neuron.update_weights(learning_rate, momentum)

In [0]:
class NeuralNetwork(ComputationalNode):

    def __init__(self):
        self.layers = []  # neuronska mreza se sastoji od slojeva neurona

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, x):  # x je vektor koji predstavlja ulaz u neuronsku mrezu
        # TODO 9: implementirati forward-pass za celu neuronsku mrezu
        # ulaz za prvi sloj neurona je x
        # ulaz za sve ostale slojeve izlaz iz prethodnog sloja
        #raise NotImplementedError('TODO 9')
        prev_layer_output=None
        for idx, layer in enumerate(self.layers):
            if idx==0:
                prev_layer_output=layer.forward(x)
            else:
                prev_layer_output=layer.forward(prev_layer_output)
        return prev_layer_output

    def backward(self, dz):
        # TODO 10: implementirati forward-pass za celu neuronsku mrezu
        # spoljasnji gradijent za izlazni sloj neurona je dz
        # spoljasnji gradijenti za ostale slojeve su izracunati gradijenti iz sledeceg sloja
        #raise NotImplementedError('TODO 10')
        next_layer_dz=None
        for idx, layer in enumerate(self.layers[::-1]):
            if idx==0:
                next_layer_dz=layer.backward(dz)
            else:
                next_layer_dz=layer.backward(next_layer_dz)
        return next_layer_dz

    def update_weights(self, learning_rate, momentum):
        # azuriranje tezina neuronske mreze je azuriranje tezina slojeva
        for layer in self.layers:
            layer.update_weights(learning_rate, momentum)

    def fit(self, X, Y, learning_rate=0.1, momentum=0.0, nb_epochs=10, shuffle=False, verbose=0):
        assert len(X) == len(Y)

        hist = []  # za plotovanje funkcije greske kroz epohe
        for epoch in range(nb_epochs):

            if shuffle:  # izmesati podatke
                random.seed(epoch)
                random.shuffle(X)
                random.seed(epoch)
                random.shuffle(Y)

            total_loss = 0.0
            for x, y in zip(X, Y):
                y_pred = self.forward(x)  # forward-pass da izracunamo izlaz
                y_target = y  # zeljeni izlaz
                grad = 0.0
                for p, t in zip(y_pred, y_target):
                    total_loss += 0.5 * (t - p) ** 2.  # funkcija greske je kvadratna greska
                    grad += -(t - p)  # gradijent funkcije greske u odnosu na izlaz
                # backward-pass da izracunamo gradijente tezina
                self.backward([[grad]])
                # azuriranje tezina na osnovu izracunatih gradijenata i koraka "learning_rate"
                self.update_weights(learning_rate, momentum)

            if verbose == 1:
                print('Epoch {0}: loss {1}'.format(epoch + 1, total_loss))

            hist.append(total_loss)

        print('Loss: {0}'.format(total_loss))
        return hist

    def predict(self, x):
        return self.forward(x)

##Model

In [0]:
nn = NeuralNetwork()

nn.add(NeuralLayer(VOCAB_SIZE, VOCAB_SIZE, 'relu'))
nn.add(NeuralLayer(VOCAB_SIZE, 125, 'relu'))
nn.add(NeuralLayer(125, 25, 'relu'))
nn.add(NeuralLayer(25, 2, 'relu'))
print(train_vectors)
history = nn.fit(train_vectors, y_train, learning_rate=0.75, nb_epochs=3, shuffle=True)

In [0]:
del nn